In [ ]:
# utils 

import numpy as np
import matplotlib.pyplot as plt

def colormap_to_rgb(cmap_name, num_colors, as_uint8=False, include_alpha=False):
    """
    Get a list of RGB or RGBA tuples from a named Matplotlib colormap.

    Parameters:
    - cmap_name (str): The name of the colormap (e.g., 'viridis', 'plasma', 'inferno').
    - num_colors (int): The number of colors to sample from the colormap.
    - as_uint8 (bool): If True, return RGB(A) values as uint8 (0-255). Otherwise, return as float (0-1).
    - include_alpha (bool): If True, return RGBA tuples. Otherwise, return RGB tuples.

    Returns:
    - colors (list of tuples): A list of RGB or RGBA tuples.
    """

    # Get the colormap from Matplotlib
    cmap = plt.get_cmap(cmap_name)

    # Generate an array of evenly spaced values in the range [0, 1]
    indices = np.linspace(0, 1, num_colors)

    # Convert colormap to a list of RGB(A) tuples
    colors = cmap(indices)

    # Convert to uint8 if required
    if as_uint8:
        colors = (colors * 255).astype(np.uint8)

    # Remove the alpha channel if not needed
    if not include_alpha:
        colors = colors[:, :3]

    # Convert to a list of tuples
    colors = [tuple(color) for color in colors]

    return colors

In [ ]:
from direct.showbase.ShowBase import ShowBase
from panda3d.core import LineSegs
from direct.task import Task

# MODE = 'HANDS_ONLY'
MODE = 'FULL_POSE'

class CubeRenderer(ShowBase):
    def __init__(self, data_reader):
        ShowBase.__init__(self)

        self.data_reader = data_reader

        # Set up the scene
        self.setup_scene()

        # Set camera position for isometric view
        self.cam.setPos(30, -30, 20)
        self.cam.lookAt(0, 0, 0)

        # Add a task to update cube positions every second
        # self.taskMgr.doMethodLater(1.0, self.update_cube_positions, 'UpdateCubesTask')
        self.taskMgr.doMethodLater(0.1, self.update_cube_positions, 'UpdateCubesTask')

    def setup_scene(self):
        # Create and add the origin axes
        self.create_axes()

        if MODE == 'HANDS_ONLY':
            num_cubes = 2
            colors = [(1, 0, 0, 1), (0, 1, 0, 1), (0, 0, 1, 1)]  # Red, Green, Blue
        elif MODE == 'FULL_POSE':
            num_cubes = 33
            colors = colormap_to_rgb('jet', num_cubes, False, True)

        # Create cubes with specified colors
        self.cubes = []
        for _ in range(num_cubes):  # Assuming 3 cubes
            cube = self.create_cube(colors.pop(0))
            self.cubes.append(cube)

    def create_axes(self):
        axes = LineSegs()
        axes.setThickness(3.0)

        # X axis (Red)
        axes.setColor(1, 0, 0, 1)
        axes.moveTo(0, 0, 0)
        axes.drawTo(5, 0, 0)

        # Y axis (Green)
        axes.setColor(0, 1, 0, 1)
        axes.moveTo(0, 0, 0)
        axes.drawTo(0, 5, 0)

        # Z axis (Blue)
        axes.setColor(0, 0, 1, 1)
        axes.moveTo(0, 0, 0)
        axes.drawTo(0, 0, 5)

        node = axes.create()
        self.render.attachNewNode(node)

    def create_cube(self, color):
        # Create a new cube
        cube = self.loader.loadModel("models/box")
        # cube.setScale(1, 1, 1)
        cube.setScale(0.5, 0.5, 0.5)
        # cube.setScale(0.25, 0.25, 0.25)

        # Set the color of the cube
        cube.setColor(*color)

        # Position the cube at the origin initially
        cube.setPos(0, 0, 0)

        cube.reparentTo(self.render)

        return cube

    def update_cube_positions(self, task):
        # Fetch the latest positions for all cubes from the data reader
        rows = self.data_reader.fetch_cube_positions()

        # Update each cube's position
        for row in rows:
            cube_id, x, y, z = row
            if cube_id < len(self.cubes):
                SF = 10
                # self.cubes[cube_id].setPos(x, y, z)
                self.cubes[cube_id].setPos(x*SF, y*SF, z*SF)
                print(f"Updated Cube {cube_id} to position ({x}, {y}, {z})")

        # Schedule the task to run again in 1 second
        return Task.again

    def cleanup(self):
        # Close the data reader connection on cleanup
        self.data_reader.close()

# Usage example:
if __name__ == "__main__":
    from sqlite_reader import SQLiteReader

    # Create a data reader instance
    data_reader = SQLiteReader()

    # Pass the data reader to the renderer
    app = CubeRenderer(data_reader)
    app.run()